In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '155_2_eda'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train_old.csv')
df_test = pd.read_csv(datadir / 'id_test.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


In [10]:
df_train.loc[:, 'has_na_sales'] = df_train.loc[:, 'NA_Sales']  > 0
df_train.loc[:, 'has_eu_sales'] = df_train.loc[:, 'EU_Sales']  > 0
df_train.loc[:, 'has_jp_sales'] = df_train.loc[:, 'JP_Sales']  > 0
df_train.loc[:, 'has_other_sales'] = df_train.loc[:, 'Other_Sales']  > 0
# df_train.to_csv(datadir / 'country_train_old.csv', index=False)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
train = df_train.copy()
test = df_test.copy()
train['is_test'] = False
test['is_test'] = True
whole = pd.concat([train, test])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

# 国のセールス

In [13]:
sales = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

In [ ]:
for s in sales:
    print(train.loc[:, s].value_counts().head())

In [ ]:
for s in sales:
    plt.figure()
    sns.distplot(train.loc[:, s])

In [ ]:
for s in sales:
    plt.figure()
    sns.distplot(train.loc[train.loc[:, s] < 10, s])

In [ ]:
for s in sales:
    plt.figure()
    sns.distplot(train.loc[train.loc[:, s] < 100, s])

In [ ]:
train.loc[:, 'has_na_sales'] = train.loc[:, 'NA_Sales']  > 0
train.loc[:, 'has_eu_sales'] = train.loc[:, 'EU_Sales']  > 0
train.loc[:, 'has_jp_sales'] = train.loc[:, 'JP_Sales']  > 0
train.loc[:, 'has_other_sales'] = train.loc[:, 'Other_Sales']  > 0

In [ ]:
test['Name'] = test['Name'].fillna('Unknown')

In [ ]:
test[test['Name'].str.contains('jp')]

In [14]:
pred_sales = [
    '156_jp_sales',
    '157_na_sales',
    '158_eu_sales',
    '159_other_sales'
]

In [ ]:
# test

In [15]:
basedir = Path('../ckpt/')
dfs = []
for pred in pred_sales:
    df = pd.read_csv(basedir / pred / (pred + '.csv'))
    dfs.append(df)
    
df_sales = pd.concat(dfs, axis=1)

In [16]:
df_sales

,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,0.024994,0.945987,0.911495,0.691802
1,0.531538,0.930108,0.946760,0.887670
2,0.136673,0.939683,0.914992,0.859691
3,0.454020,0.945443,0.971069,0.949068
4,0.600900,0.922788,0.963546,0.951913
...,...,...,...,...
8355,0.921956,0.253061,0.222631,0.210761
8356,0.902996,0.261210,0.244244,0.248479
8357,0.006802,0.181673,0.938938,0.071860
8358,0.011147,0.203807,0.892700,0.078313


In [17]:
df_test_sales = pd.concat([df_test, df_sales], axis=1)
df_test_sales.to_csv(datadir / 'country_prob_test_old.csv', index=False)
df_test_sales.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,Hitman 2: Silent Assassin,XB,NaN,Action,Eidos Interactive,84.0,23.0,8,19.0,Io Interactive,M,8359,0.024994,0.945987,0.911495,0.691802
1,Legacy of Kain: Soul Reaver,PS,NaN,Action,Eidos Interactive,91.0,17.0,9,132.0,Crystal Dynamics,T,8360,0.531538,0.930108,0.946760,0.887670
2,Metal Gear Solid 2: Substance,XB,NaN,Action,Konami Digital Entertainment,87.0,28.0,8.5,39.0,KCEJ,M,8361,0.136673,0.939683,0.914992,0.859691
3,Silent Hill: Homecoming,X360,NaN,Action,Konami Digital Entertainment,70.0,54.0,6.9,180.0,Double Helix Games,M,8362,0.454020,0.945443,0.971069,0.949068
4,Silent Hill: Homecoming,PS3,NaN,Action,Konami Digital Entertainment,71.0,41.0,6.9,143.0,Double Helix Games,M,8363,0.600900,0.922788,0.963546,0.951913


In [ ]:
# training

In [18]:
basedir = Path('../ckpt/')
dfs = []
for pred in pred_sales:
    df = pd.read_csv(basedir / pred / ('train_' + pred + '.csv'))
    dfs.append(df)
    
df_sales = pd.concat(dfs, axis=1)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
df_sales

,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
0,0.022711,0.953149,0.930630,0.952401
1,0.022196,0.950683,0.942255,0.944289
2,0.020971,0.953247,0.909297,0.937944
3,0.021234,0.848037,0.889520,0.388019
4,0.021598,0.951913,0.922003,0.935384
...,...,...,...,...
8354,0.008785,0.207528,0.973386,0.413671
8355,0.007776,0.157787,0.937090,0.078177
8356,0.962374,0.139562,0.066496,0.036658
8357,0.927160,0.438481,0.288954,0.338038


In [20]:
has_sales = ['has_jp_sales', 'has_na_sales', 'has_eu_sales', 'has_other_sales']

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
df_train_sales = df_train.copy()
for c in has_sales:
    df_train_sales.loc[:, c] = df_sales.loc[:, c]
df_train_sales.to_csv(datadir / 'country_prob_train_old.csv', index=False)

In [22]:
df_train_sales.tail()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Critic_Count,User_Score,User_Count,Developer,Rating,id,has_na_sales,has_eu_sales,has_jp_sales,has_other_sales
8354,Stellaris,PC,2016.0,Strategy,Paradox Interactive,0,4,0,0,4,...,57.0,8,569.0,Paradox Development Studio,NaN,8354,0.207528,0.973386,0.008785,0.413671
8355,Total War Attila: Tyrants & Kings,PC,2016.0,Strategy,Koch Media,0,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,8355,0.157787,0.937090,0.007776,0.078177
8356,Brothers Conflict: Precious Baby,PSV,2017.0,Action,Idea Factory,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,8356,0.139562,0.066496,0.962374,0.036658
8357,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017.0,Role-Playing,Sega,0,0,4,0,4,...,NaN,NaN,NaN,NaN,NaN,8357,0.438481,0.288954,0.927160,0.338038
8358,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017.0,Role-Playing,Sega,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,8358,0.224404,0.172157,0.941215,0.131783


In [23]:
df_tmp = []
for c in has_sales:
    df_tmp.append(df_train.loc[:, c] == (df_train_sales.loc[:, c] > 0.55))
df_tmp = pd.concat(df_tmp, axis=1)
np.mean(df_tmp.values)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9293575786577342

In [24]:
df_test_det = df_test_sales.copy()
for c in has_sales:
    df_test_det.loc[:, c] = df_test_sales.loc[:, c] > 0.55
df_test_det.to_csv(datadir / 'country_test_old.csv', index=False)
df_test_det.tail()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,has_jp_sales,has_na_sales,has_eu_sales,has_other_sales
8355,SD Gundam G Generation Genesis,PSV,2016.0,Strategy,Namco Bandai Games,NaN,NaN,NaN,NaN,NaN,NaN,16714,True,False,False,False
8356,SD Gundam G Generation Genesis,PS4,2016.0,Strategy,Namco Bandai Games,NaN,NaN,NaN,NaN,NaN,NaN,16715,True,False,False,False
8357,Battle Worlds: Kronos,PC,2016.0,Strategy,Nordic Games,71.0,15.0,7.1,71.0,KING Art,NaN,16716,False,False,True,False
8358,Codename: Panzers Complete Collection,PC,2016.0,Strategy,Nordic Games,NaN,NaN,NaN,NaN,NaN,NaN,16717,False,False,True,False
8359,Imagine: Makeup Artist,DS,2020.0,Simulation,Ubisoft,NaN,NaN,tbd,NaN,Ubisoft,E,16718,False,True,False,False


In [ ]:
df_train_sales.loc[:, ['has_jp_sales', 'has_na_sales', 'has_eu_sales', 'has_other_sales']].head()

In [ ]:
df_train.loc[:, ['has_jp_sales', 'has_na_sales', 'has_eu_sales', 'has_other_sales']].head()

In [ ]:
tpr

In [ ]:
df_train.loc[df_train['Name'].str.contains('Mario'), :]

In [ ]:
df_test.loc[df_test['Name'].fillna('NaN').str.contains('Mario'), :]

In [ ]:
df_test